In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle competitions download -c covid-19-mrna-vaccine-degradation-41

 84% 7.00M/8.29M [00:01<00:00, 8.45MB/s]
100% 8.29M/8.29M [00:01<00:00, 5.90MB/s]


In [3]:
!unzip covid-19-mrna-vaccine-degradation-41.zip

Archive:  covid-19-mrna-vaccine-degradation-41.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [5]:
def mcrmse_loss(y_true, y_pred, N=3):

    assert len(y_true) == len(y_pred)
    n = len(y_true)
    return np.sum(np.sqrt(np.sum((y_true - y_pred)**2, axis=0)/n)) / N

In [6]:
train = pd.read_csv('train.csv')
# train.T

In [7]:
test = pd.read_csv('test.csv')
test

,id,id_seqpos,sequence,structure,predicted_loop_type,reactivity_error,deg_error_Mg_pH10,deg_pH10,deg_error_pH10,deg_error_Mg_50C,...,b4_structure,a4_structure,b4_predicted_loop_type,a4_predicted_loop_type,b5_sequence,a5_sequence,b5_structure,a5_structure,b5_predicted_loop_type,a5_predicted_loop_type
0,id_001f94081,id_001f94081_0,G,.,E,0.1359,0.2613,2.3375,0.2631,0.1501,...,-1,.,-1,E,-1,A,-1,(,-1,S
1,id_001f94081,id_001f94081_4,A,.,E,0.1314,0.1798,0.2635,0.1000,0.1369,...,.,(,E,S,-1,C,-1,(,-1,S
2,id_001f94081,id_001f94081_8,U,(,S,0.0756,0.1056,0.0530,0.0517,0.0705,...,.,.,E,H,A,U,.,.,E,H
3,id_001f94081,id_001f94081_9,C,(,S,0.1087,0.1896,0.5348,0.1474,0.1588,...,(,.,S,H,A,A,.,.,E,H
4,id_001f94081,id_001f94081_13,U,.,H,0.1388,0.1810,0.7699,0.1417,0.1382,...,(,.,S,H,U,G,(,),S,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32635,id_fff546103,id_fff546103_39,A,(,S,0.0091,0.0344,0.0140,0.0303,0.0284,...,(,(,S,S,C,A,(,.,S,H
32636,id_fff546103,id_fff546103_52,A,),S,0.0237,0.0390,0.0806,0.0529,0.0361,...,),),S,S,G,G,),),S,S
32637,id_fff546103,id_fff546103_62,A,.,B,0.0655,0.0672,0.5538,0.1022,0.0694,...,.,),M,S,G,C,),),S,S
32638,id_fff546103,id_fff546103_65,A,),S,0.0218,0.0391,0.1642,0.0611,0.0520,...,),(,S,S,U,C,),(,S,S


In [8]:
categorical_columns = [ 'predicted_loop_type', 'b1_structure', 'a1_structure',
                       'b2_structure', 'a2_structure', 'b3_structure', 'a3_structure', 'b4_structure',
                       'a4_structure', 'b5_structure', 'a5_structure', 'b1_predicted_loop_type',
                       'a1_predicted_loop_type', 'b2_predicted_loop_type', 'a2_predicted_loop_type',
                       'b3_predicted_loop_type', 'a3_predicted_loop_type', 'b4_predicted_loop_type',
                       'a4_predicted_loop_type', 'b5_predicted_loop_type', 'a5_predicted_loop_type','b1_sequence', 'a1_sequence', 'b2_sequence', 'a2_sequence', 'b3_sequence', 'a3_sequence', 'b4_sequence', 'a4_sequence','sequence', 'structure', 'b5_sequence', 'a5_sequence']

In [9]:
# Use get_dummies to perform one-hot encoding
train = pd.get_dummies(train, columns=categorical_columns)

In [10]:
train.shape

(130560, 181)

In [11]:
test = pd.get_dummies(test, columns=categorical_columns)

In [12]:
train

,id,id_seqpos,reactivity,reactivity_error,deg_Mg_pH10,deg_error_Mg_pH10,deg_pH10,deg_error_pH10,deg_Mg_50C,deg_error_Mg_50C,...,structure_.,b5_sequence_-1,b5_sequence_A,b5_sequence_C,b5_sequence_G,b5_sequence_U,a5_sequence_A,a5_sequence_C,a5_sequence_G,a5_sequence_U
0,id_159f0ff46,id_159f0ff46_44,1.7786,0.1195,0.7346,0.0790,0.3680,0.0851,0.7228,0.0792,...,1,0,0,1,0,0,1,0,0,0
1,id_d73546f4e,id_d73546f4e_38,0.4397,0.1033,0.1630,0.0799,0.0878,0.0878,0.1100,0.0683,...,1,0,0,1,0,0,0,1,0,0
2,id_f3037fedf,id_f3037fedf_34,0.1120,0.1375,0.0000,0.0744,0.1835,0.1969,0.0933,0.1163,...,0,0,1,0,0,0,0,0,1,0
3,id_468300749,id_468300749_12,0.0622,0.0471,0.1179,0.1075,0.0657,0.0615,0.0548,0.0506,...,0,0,1,0,0,0,0,0,0,1
4,id_69e3878a2,id_69e3878a2_28,0.1378,0.1352,0.4069,0.2168,0.3375,0.2623,0.2378,0.1569,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130555,id_796128149,id_796128149_27,0.1244,0.0393,0.2672,0.0728,0.1479,0.0493,0.1921,0.0481,...,0,0,1,0,0,0,1,0,0,0
130556,id_8a466e770,id_8a466e770_39,0.0201,0.0351,0.1271,0.0583,0.0580,0.0660,0.0245,0.0419,...,1,0,1,0,0,0,0,0,0,1
130557,id_998669e76,id_998669e76_52,0.4597,0.0601,0.2014,0.0515,0.2838,0.0793,0.2042,0.0528,...,1,0,0,0,0,1,0,0,0,1
130558,id_6fe4a1abc,id_6fe4a1abc_45,0.3634,0.1063,0.1928,0.0738,0.1885,0.1276,0.1627,0.0773,...,1,0,0,0,1,0,0,0,1,0


In [13]:
train = train.drop(['id', 'id_seqpos'],axis=1)

In [14]:
# train = train.drop(['sequence',	'structure'	],axis=1)
# test = test.drop(['sequence',	'structure'	],axis=1)

In [15]:
# Split data into features (X) and target variables (y)
X = train.drop(['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C'], axis=1)
y = train[['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']]

# # Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
X.shape,y.shape


((130560, 176), (130560, 3))

In [17]:
wanted = test.drop(['id', 'id_seqpos'], axis=1)

In [18]:


from sklearn.decomposition import PCA
# Apply PCA for dimensionality reduction
pca = PCA(n_components=100)  # Experiment with the number of components
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
wanted = pca.transform(wanted)




In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb


# Train an XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=600,random_state=42)
model.fit(X_train, y_train)

# Make predictions using the trained model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r_squared}")

In [ ]:
mcrmse_loss(y_test, y_pred, N=3)

***

In [38]:
y_pred_test = model.predict(wanted)

In [39]:
y_pred_test

array([[0.4605671 , 0.8513384 , 0.44921425],
       [0.6572668 , 0.55398345, 0.6143041 ],
       [0.23303266, 0.1826492 , 0.17339963],
       ...,
       [0.51300275, 0.5339817 , 0.47906148],
       [0.03716465, 0.20796941, 0.23215109],
       [0.01856728, 0.21439107, 0.23510595]], dtype=float32)

In [23]:
submission_df = pd.read_csv('/content/sample_submission.csv')
submission_df

,id_seqpos,reactivity,deg_Mg_pH10,deg_Mg_50C
0,id_001f94081_0,0.0,0.0,0.0
1,id_001f94081_4,0.0,0.0,0.0
2,id_001f94081_8,0.0,0.0,0.0
3,id_001f94081_9,0.0,0.0,0.0
4,id_001f94081_13,0.0,0.0,0.0
...,...,...,...,...
32635,id_fff546103_39,0.0,0.0,0.0
32636,id_fff546103_52,0.0,0.0,0.0
32637,id_fff546103_62,0.0,0.0,0.0
32638,id_fff546103_65,0.0,0.0,0.0


In [40]:
submission_df = pd.DataFrame({
    'id_seqpos': test['id_seqpos'],  # Assuming 'test' is your testing data
    'reactivity': y_pred_test[:, 0],  # Predicted values for the 'reactivity' target
    'deg_Mg_pH10': y_pred_test[:, 1],  # Predicted values for the 'deg_Mg_pH10' target
    'deg_Mg_50C': y_pred_test[:, 2]  # Predicted values for the 'deg_Mg_50C' target
})

In [41]:
submission_df

,id_seqpos,reactivity,deg_Mg_pH10,deg_Mg_50C
0,id_001f94081_0,0.460567,0.851338,0.449214
1,id_001f94081_4,0.657267,0.553983,0.614304
2,id_001f94081_8,0.233033,0.182649,0.173400
3,id_001f94081_9,0.282330,0.855466,1.016176
4,id_001f94081_13,1.153072,1.225994,1.055729
...,...,...,...,...
32635,id_fff546103_39,0.006907,0.060117,0.043152
32636,id_fff546103_52,0.029104,0.163248,0.116644
32637,id_fff546103_62,0.513003,0.533982,0.479061
32638,id_fff546103_65,0.037165,0.207969,0.232151


In [42]:
submission_df.to_csv('submission_onehot_600.csv', index=False)